<a href="https://colab.research.google.com/github/sidms24/internship/blob/main/Influenza/ili_and_climate/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [168]:
!wget -q -O "climate.csv.gz" https://www.data.gouv.fr/api/1/datasets/r/c79aaafe-8017-4d2b-8884-57b5391da5bc
!wget -q -O "ili.csv"  https://www.sentiweb.fr/datasets/all/inc-3-PAY.csv

In [ ]:
!gunzip climate.csv.gz

gzip: climate.csv already exists; do you wish to overwrite (y or n)? y


In [ ]:
import pandas as pd

In [181]:
df = pd.read_csv('climate.csv', sep = ';')
df_ili = pd.read_csv('ili.csv', skiprows=1)
df = df[['NUM_POSTE',
 'NOM_USUEL',
 'LAT',
 'LON',
 'ALTI',
 'AAAAMMJJ',
 'RR',
 'QRR',
 'TN',
 'QTN',
 'TX',
 'QTX',]]


In [182]:
df["AAAAMMJJ"] = pd.to_datetime(df["AAAAMMJJ"], format="%Y%m%d")
df = df.sort_values(by=['AAAAMMJJ'])
df = df.dropna()
df['week'] = df['AAAAMMJJ'].dt.year.astype(str)+ df['AAAAMMJJ'].dt.isocalendar().week.astype(str).apply(lambda x: '0' + x if len(x) == 1 else x)

In [183]:
df = df.groupby(['NUM_POSTE',
 'NOM_USUEL',
 'LAT',
 'LON',
 'ALTI',
  'week']).agg({'RR':'mean', 'QRR':'sum', 'TN':'min', 'QTN':'sum', 'TX':'max', 'QTX':'sum'})
df.sort_values(by=['week'])
df.reset_index(inplace=True)
df.sort_values(by=['week'], inplace = True)
df['week'] = df['week'].astype(int)
df.rename(columns = {'NOM_USUEL':'region',
                     'RR':'rainfall_rate',
                     'TN':'temp_min',
                     'TX':'temp_max'}, inplace = True)
df_ili.drop(columns = ['geo_name', 'geo_insee'], inplace = True)

In [184]:
df = pd.merge(df, df_ili, on = ['week'])
df.dropna(inplace = True)

In [190]:
df

,NUM_POSTE,region,LAT,LON,ALTI,week,rainfall_rate,QRR,temp_min,QTN,temp_max,QTX,indicator,inc,inc_low,inc_up,inc100,inc100_low,inc100_up
0,1014001,ARBENT,46.266667,5.650000,535,198444,1.871429,7.0,-2.0,7.0,19.1,7.0,3,68422,20056.0,116788.0,125,37.0,213.0
1,1053003,BOURG EN BRESSE,46.215000,5.258333,250,198444,1.914286,7.0,0.3,7.0,19.1,7.0,3,68422,20056.0,116788.0,125,37.0,213.0
2,1210001,LELEX,46.301667,5.941667,930,198444,0.928571,7.0,1.0,7.0,17.5,7.0,3,68422,20056.0,116788.0,125,37.0,213.0
3,1262003,MONTLUEL,45.900000,5.016667,280,198444,2.385714,7.0,1.0,7.0,18.0,7.0,3,68422,20056.0,116788.0,125,37.0,213.0
4,1082001,CHANAY,46.003333,5.788333,480,198444,0.800000,7.0,2.5,7.0,17.0,7.0,3,68422,20056.0,116788.0,125,37.0,213.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43753,1414001,SUTRIEU,45.916167,5.624667,878,202352,1.950000,8.0,-1.6,8.0,14.8,8.0,3,115446,103738.0,127154.0,174,156.0,192.0
43754,1367002,SAINT JULIEN SUR REYSSOUZE,46.403167,5.109833,189,202352,0.925000,8.0,-2.6,8.0,17.8,8.0,3,115446,103738.0,127154.0,174,156.0,192.0
43755,1247003,MIJOUX,46.378833,6.019333,1002,202352,3.387500,8.0,-5.7,8.0,13.9,8.0,3,115446,103738.0,127154.0,174,156.0,192.0
43756,1235001,MARLIEUX,46.038500,5.044333,280,202352,0.875000,8.0,-2.3,8.0,17.0,8.0,3,115446,103738.0,127154.0,174,156.0,192.0
